#### Importing Required Libraries

In [195]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


#### Loading the DataSet

In [196]:
data = pd.read_csv("Bengaluru_House_Data.csv")
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


#### Checking D-Types & Shape

In [197]:
d = data.shape
d

(13320, 9)

In [198]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [199]:
#Let's do Value_counts for each Column
for column in data.columns:
    print(data[column].value_counts())
    print("*"*30)

Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: area_type, dtype: int64
******************************
Ready To Move    10581
18-Dec             307
18-May             295
18-Apr             271
18-Aug             200
                 ...  
15-Aug               1
17-Jan               1
16-Nov               1
16-Jan               1
14-Jul               1
Name: availability, Length: 81, dtype: int64
******************************
Whitefield                        540
Sarjapur  Road                    399
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       234
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: location, Length: 1305, dtype: int64
******************************
2 

## Treating the Null Values

In [200]:
null_cnt = data.isnull().sum(axis=0).sort_values(ascending=False)
percent_null_cnt = round((data.isnull().sum(axis=0)/data.shape[0]*100).sort_values(ascending=False),2)

missing_data = pd.concat([null_cnt, percent_null_cnt], axis=1, keys = ['Total', 'Percentage of Missing Values'])
missing_data 

,Total,Percentage of Missing Values
society,5502,41.31
balcony,609,4.57
bath,73,0.55
size,16,0.12
location,1,0.01
area_type,0,0.00
availability,0,0.00
total_sqft,0,0.00
price,0,0.00


In [201]:
for column in data.columns:
    print(column,data[column].nunique())
    print("*"*20)

area_type 4
********************
availability 81
********************
location 1305
********************
size 31
********************
society 2688
********************
total_sqft 2117
********************
bath 19
********************
balcony 4
********************
price 1994
********************


* Society is having 41percent null values
* Area type is not going to have any impact on my model
* Availabiltiy is gone change based on many factors 
* Balcony is not going to impact my model since the square foot is same with or without it 

* Based on the above criteria we are dropping all of these columns

In [203]:
data.drop(columns=["society","area_type","availability","balcony"],inplace=True)

In [204]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13319 non-null  object 
 1   size        13304 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


* Three columns are having null values
* Even though the bath (bathroom) is float but considering as categorical column and imputed with mode

In [205]:
colm  = ["size","bath","location"]
for i in colm:
    data[i] = data[i].fillna(data[i].mode()[0])

In [206]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13320 non-null  object 
 1   size        13320 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13320 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


* All the null Values has been Imputed. 

## Data Cleaning

In [208]:
data["size"].value_counts()

2 BHK         5215
3 BHK         4310
4 Bedroom      826
4 BHK          591
3 Bedroom      547
1 BHK          538
2 Bedroom      329
5 Bedroom      297
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
11 Bedroom       2
10 BHK           2
14 BHK           1
13 BHK           1
12 Bedroom       1
27 BHK           1
43 Bedroom       1
16 BHK           1
19 BHK           1
18 Bedroom       1
Name: size, dtype: int64

* In the size column the same values are repeated as "bedroom","BHK","RK"
* so we are extract only the numerical value  and storing it in a new column as "BHK"

In [210]:
data["BHK"] = data["size"].str.split().str.get(0).astype(int)

In [226]:
data["BHK"]

0        2
1        4
2        3
3        3
4        2
        ..
13315    5
13316    4
13317    2
13318    4
13319    1
Name: BHK, Length: 13320, dtype: int32

In [211]:
data["total_sqft"].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

* In the total_sqft column some values are given in the range(ex:'1133 - 1384')
* so we are removing this and the datatype is in object that is also change to float

In [213]:
def range_convertion(x):
    temp = x.split('-')
    if len(temp) == 2:
        return(float(temp[0])+float(temp[1]))/2
    try :
        return float(x)
    except:
        return None

In [214]:
data["total_sqft"] = data["total_sqft"].apply(range_convertion)

* here we handle the value which in the range by adding it and divided by 2
* so that we can get the in between value in the range.
* and also there is some values with not int or float so convert into none value

In [216]:
data.head()

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2


In [217]:
data["location"].value_counts()

Whitefield                        541
Sarjapur  Road                    399
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       234
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: location, Length: 1305, dtype: int64

In [218]:
data["location"].nunique()

1305

In [219]:
data["location"] = data["location"].apply(lambda x: x.strip())
data["location"].nunique()

1294

* here we can able to se that after doing the strip the no of unique 
* values got to reduce from 1305 to 1294 which implies that same values 
* are filled with space and considering as unique values

In [221]:
location_count = data["location"].value_counts()
location_count_less_10 = location_count[location_count<=10]
location_count_less_10

Dairy Circle                      10
Nagappa Reddy Layout              10
Basapura                          10
1st Block Koramangala             10
Sector 1 HSR Layout               10
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: location, Length: 1053, dtype: int64

* here totally 1305 unique locations are there.For Better analysis we are taking the location which is appeared more than 10
* which is 252 locations and 1053 location are less than the 10 times appeared. So we are going to change the 1053 values as others

In [ ]:
data["location"] = data["location"].apply(lambda x: "other" if x in location_count_less_10 else x)
data["location"].value_counts()

* from 1305 locations we reduced to 242 unique location 

## Price Per Square Feet

* here by doing a Feature Engineering we are creating a new column
* which describe the Price Per square_feet(our price column is in Lakhs)
* so multiplied by lakhs and divided by total_sqft so that we can get the price per square feet value

In [153]:
data

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2
...,...,...,...,...,...,...
13315,Whitefield,5 Bedroom,3453.0,4.0,231.00,5
13316,other,4 BHK,3600.0,5.0,400.00,4
13317,Raja Rajeshwari Nagar,2 BHK,1141.0,2.0,60.00,2
13318,Padmanabhanagar,4 BHK,4689.0,4.0,488.00,4


In [154]:
data["price_per_sqft"] = data["price"] * 100000 / data["total_sqft"]

## Outlier Treatment

In [155]:
 data.describe()

,total_sqft,bath,price,BHK,price_per_sqft
count,13274.000000,13320.000000,13320.000000,13320.000000,1.327400e+04
mean,1559.626694,2.688814,112.565627,2.802778,7.907501e+03
std,1238.405258,1.338754,148.971674,1.294496,1.064296e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.266865e+03
50%,1276.000000,2.000000,72.000000,3.000000,5.434306e+03
75%,1680.000000,3.000000,120.000000,3.000000,7.311746e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


* calculating the total_sqft per BHK 

In [158]:
(data["total_sqft"]/data["BHK"]).describe()

count    13274.000000
mean       575.074878
std        388.205175
min          0.250000
25%        473.333333
50%        552.500000
75%        625.000000
max      26136.000000
dtype: float64

* from this we can infer that minimum value of sq_ft per BHK is  0.25
  this is totally not possible so that we are going to filter the 
* total_sqft per BHK greater than 300.considering 300sq_ft as the minimum value.

In [160]:
data_new = data[((data["total_sqft"]/data["BHK"])  >= 300)]
data_new.describe()

,total_sqft,bath,price,BHK,price_per_sqft
count,12530.000000,12530.000000,12530.000000,12530.000000,12530.000000
mean,1594.564544,2.559537,111.382401,2.650838,6303.979357
std,1261.271296,1.077938,152.077329,0.976678,4162.237981
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1116.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.117647
75%,1700.000000,3.000000,115.000000,3.000000,6916.666667
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


* here our minimum value of total_sqft came to 300

In [163]:
print(f"Before Removing the Minimum total_sqft the shape is {data.shape} and after removing outlier the shape is {data_new.shape} i.e {data.shape[0]-data_new.shape[0]} records removed ")

Before Removing the Minimum total_sqft the shape is (13320, 7) and after removing outlier the shape is (12530, 7) i.e 790 records removed 


* on seeing the Price_per_sqft max Value(176470.588) surely it is an outlier so we need to remove this
  Here we cant able to remove the outliers by using iqr method. 
* Because based on the each location price_per_sqft will change
  so that we are calculating the mean and std on location wise by using "groupby" using that we are removing the outliers which are all away from one std

### Bivariate Approach

* handling the Outliers in the Price_per_sqft by using using location

In [165]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby("location"):
        m = np.mean(subdf.price_per_sqft)

        st = np.std(subdf.price_per_sqft)

        gen_df = subdf[(subdf.price_per_sqft > (m - st)) & (subdf.price_per_sqft <= (m + st))]
        df_output = pd.concat([df_output,gen_df],ignore_index = True)
    return df_output
data_new = remove_outliers_sqft(data_new)
data_new.describe()


,total_sqft,bath,price,BHK,price_per_sqft
count,10301.000000,10301.000000,10301.000000,10301.000000,10301.000000
mean,1508.440608,2.471702,91.286372,2.574896,5659.062876
std,880.694214,0.979449,86.342786,0.897649,2265.774749
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4244.897959
50%,1286.000000,2.000000,67.000000,2.000000,5175.600739
75%,1650.000000,3.000000,100.000000,3.000000,6428.571429
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [166]:
print(f"Before Removing the Outlier the shape is {data.shape} and after removing outlier the shape is {data_new.shape} i.e {data.shape[0]-data_new.shape[0]} records removed in this Price_per_sqft outlier removal")

Before Removing the Outlier the shape is (13320, 7) and after removing outlier the shape is (10301, 7) i.e 3019 records removed in this Price_per_sqft outlier removal


### Multivariate Aproach

* we are removing the outlier in BHK by using location and Price_per_sqft

###### Approach for the below code 

* Based on the Each Location for the "n" BHK minimum price_per_sqft should be greater than the 
  mean of "n-1"th price_per_sqft(for ex.if it is 3BHK the minimum price_per_sqft should be 
  greater than the mean price_per_sqft of 2bhk)  
* For taking the mean  the count of BHK should be atleast 6 on each location. So that the mean will be reliable.
  If the above condition is True we removed the record.

In [176]:
def bhk_outlier_removal(df):
    exclude_indices = np.array([])
    for location,location_df in df.groupby("location"):
        BHK_stats = {}
        for BHK,BHK_df in location_df.groupby("BHK"):
            BHK_stats[BHK] = {
                "mean" : np.mean(BHK_df.price_per_sqft),
                "std" : np.std(BHK_df.price_per_sqft),
                "count" : BHK_df.shape[0]
            }
        
        for BHK, BHK_df in location_df.groupby("BHK"):
            stats = BHK_stats.get(BHK-1)
            if stats and stats["count"]>5:
                exclude_indices = np.append(exclude_indices, BHK_df[BHK_df.price_per_sqft < (stats["mean"])].index.values)
    return df.drop(exclude_indices,axis="index")

In [177]:
data_new_1 = bhk_outlier_removal(data_new)

In [178]:
print(f"Before Removing the Outlier the shape is {data_new.shape} and after removing outlier the shape is {data_new_1.shape} i.e {data_new.shape[0]-data_new_1.shape[0]} records removed in this BHK outlier removal")

Before Removing the Outlier the shape is (10301, 7) and after removing outlier the shape is (7361, 7) i.e 2940 records removed in this BHK outlier removal


In [179]:
data_new_1.info()      

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7361 entries, 0 to 10300
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location        7361 non-null   object 
 1   size            7361 non-null   object 
 2   total_sqft      7361 non-null   float64
 3   bath            7361 non-null   float64
 4   price           7361 non-null   float64
 5   BHK             7361 non-null   int32  
 6   price_per_sqft  7361 non-null   float64
dtypes: float64(4), int32(1), object(2)
memory usage: 431.3+ KB


In [181]:
data_new_1

,location,size,total_sqft,bath,price,BHK,price_per_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668
...,...,...,...,...,...,...,...
10292,other,2 BHK,1200.0,2.0,70.0,2,5833.333333
10293,other,1 BHK,1800.0,1.0,200.0,1,11111.111111
10296,other,2 BHK,1353.0,2.0,110.0,2,8130.081301
10297,other,1 Bedroom,812.0,1.0,26.0,1,3201.970443


In [187]:
print(f"From {d} after removing outliers we got {data_new_1.shape}i.e {d[0] - data_new_1.shape[0]}  records we removed")

From (13320, 7) after removing outliers we got (7361, 7)i.e 5959  records we removed


* we are dropping the size column and Price_per_sqft column for doing ML the price_per_sqft is used only for removing the outliers

In [189]:
data_new_1.drop(columns = ["size","price_per_sqft"],inplace = True)

In [249]:
data_new_1.head()

,location,total_sqft,bath,price,BHK
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


### Encoding

In [250]:
pd.get_dummies(data=data_new_1,columns = ["location"],drop_first = True)

,total_sqft,bath,price,BHK,location_1st Phase JP Nagar,location_2nd Phase Judicial Layout,location_2nd Stage Nagarbhavi,location_5th Block Hbr Layout,location_5th Phase JP Nagar,location_6th Phase JP Nagar,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
0,2850.0,4.0,428.0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,194.0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,235.0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1200.0,2.0,130.0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1235.0,2.0,148.0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10292,1200.0,2.0,70.0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10293,1800.0,1.0,200.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10296,1353.0,2.0,110.0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10297,812.0,1.0,26.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Cleaned Data

In [190]:
data_new_1.head()

,location,total_sqft,bath,price,BHK
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [ ]:
#we are saving the cleaned data

In [191]:
data_new_1.to_csv("Cleaned_Data.csv")